In [2]:
import pandas as pd
import numpy as np

In [3]:
sub_filtered = pd.read_csv('/Users/vivianyan/Desktop/NYU_Capstone_Project/data_preprocessing/valid_full_sub.csv')
sub_cs = sub_filtered[sub_filtered.BRAND=='ChillStream']
sub_cs[sub_cs.ID=='020TBXH154']

,Unnamed: 0,ID,BRAND,SUB_STATUS,TERM,AMOUNT,PARTITION_DATE,YEAR_MONTH,VALID_TRANSITION
2736,4480,020TBXH154,ChillStream,expired,monthly,24.99,2022-02-16,2022-02,True
2737,4481,020TBXH154,ChillStream,paid,monthly,24.99,2022-04-29,2022-04,True
2738,4484,020TBXH154,ChillStream,expired,monthly,24.99,2022-11-06,2022-11,True
2739,4485,020TBXH154,ChillStream,paid,monthly,25.00,2023-01-30,2023-01,True


In [4]:
def calc_rev(df, start, end):
    full_agg_data = []
    full_grouped = df.groupby(['ID', 'BRAND'])

    for (id_brand, group) in full_grouped:
        group = group.reset_index(drop=True)

        start_date = start
        end_date = end
        sub_detail = ''
        revenue = 0
        amount = 0
        
        for i, row in group.iterrows():
            sub_status = row['SUB_STATUS']
            partition_date = row['PARTITION_DATE']
                
            if sub_status in ['trial', 'discounted']:
                start_date = partition_date
                sub_detail = row['SUB_STATUS'] + ' ' + row['TERM']
                amount = row['AMOUNT']
                term = row['TERM']

            elif sub_status == 'paid':
                if amount != 0:
                    duration = (partition_date - start_date).days
                    if row['TERM'] == 'monthly':
                        revenue += np.ceil(duration / 30) * amount
                    else:
                        revenue += np.ceil(duration / 365) * amount

                start_date = partition_date
                sub_detail = row['SUB_STATUS'] + ' ' + row['TERM']
                amount = row['AMOUNT']
                term = row['TERM']
                    
            else:
                end_date = partition_date
                duration = (end_date - start_date).days
                if start_date == start:
                    sub_detail = row['SUB_STATUS'] + ' ' + row['TERM']
                    if duration == 0:
                        end_date = end
                        continue
                    else:
                        amount = row['AMOUNT']
                
                if row['TERM'] == 'monthly':
                    revenue += np.ceil(duration / 30) * amount
                else:
                    revenue += np.ceil(duration / 365) * amount

                full_agg_data.append({
                    'ID': id_brand[0],
                    'BRAND': id_brand[1],
                    'SUB_DETAIL': sub_detail,
                    'AMOUNT': amount,
                    'START_DATE': start_date,
                    'END_DATE': end_date,
                    'DURATION': duration,
                    'REVENUE': revenue
                })            
                start_date = partition_date
                end_date = end
                amount = 0

        if amount != 0:
            duration = (end_date - start_date).days
            if term == 'monthly':
                revenue = np.ceil(duration / 30) * amount
            else:
                revenue = np.ceil(duration / 365) * amount
            
            full_agg_data.append({
                        'ID': id_brand[0],
                        'BRAND': id_brand[1],
                        'SUB_DETAIL': sub_detail,
                        'AMOUNT': amount,
                        'START_DATE': start_date,
                        'END_DATE': end_date,
                        'DURATION': duration,
                        'REVENUE': revenue
                    })

    full_agg = pd.DataFrame(full_agg_data)
    return full_agg

In [5]:
sub_filtered['PARTITION_DATE'] = pd.to_datetime(sub_filtered['PARTITION_DATE'])

sub_rev = calc_rev(sub_filtered, start=pd.to_datetime('2022-01-01'), end=pd.to_datetime('2023-12-31'))
sub_rev

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
0,000AHHU956,RetroReel,expired monthly,30.00,2022-01-01,2022-02-17,47,60.00
1,000AHHU956,RetroReel,paid monthly,29.99,2022-05-21,2023-10-28,525,599.82
2,000AHHU956,RetroReel,trial monthly,11.18,2023-11-05,2023-12-31,56,22.36
3,000BRHY264,CineQuest,paid annual,60.00,2022-05-23,2023-10-17,512,120.00
4,000DJKK225,CineQuest,expired monthly,12.00,2022-01-01,2022-04-15,104,48.00
...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,2023-10-01,2023-12-31,91,31.92
89541,999ZGDS170,CineQuest,expired annual,60.00,2022-01-01,2023-03-02,425,120.00
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,2023-12-31,39,59.99
89543,999ZIAU985,RetroReel,paid monthly,30.01,2022-01-28,2023-12-31,702,720.24


In [18]:
sub_rev.to_csv('/Users/vivianyan/Desktop/NYU_Capstone_Project/recalc_revenue/full_rev.csv')

In [6]:
start_2022, end_2022 = pd.to_datetime('2022-01-01'), pd.to_datetime('2022-12-31')
sub_2022 = sub_filtered[(sub_filtered['PARTITION_DATE'] >= start_2022) & (sub_filtered['PARTITION_DATE'] <= end_2022)]
sub_rev_2022 = calc_rev(sub_2022, start=start_2022, end=end_2022)
sub_rev_2022

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
0,000AHHU956,RetroReel,expired monthly,30.00,2022-01-01,2022-02-17,47,60.00
1,000AHHU956,RetroReel,paid monthly,29.99,2022-05-21,2022-12-31,224,239.92
2,000BRHY264,CineQuest,paid annual,60.00,2022-05-23,2022-12-31,222,60.00
3,000DJKK225,CineQuest,expired monthly,12.00,2022-01-01,2022-04-15,104,48.00
4,000DJKK225,CineQuest,paid monthly,12.00,2022-07-17,2022-12-31,167,72.00
...,...,...,...,...,...,...,...,...
60173,999WJDL067,ChillStream,paid monthly,25.01,2022-11-25,2022-12-31,36,50.02
60174,999WMMD928,LimeLight,paid annual,80.00,2022-05-31,2022-12-31,214,80.00
60175,999WVJU110,Pulse,expired monthly,8.00,2022-01-01,2022-04-21,110,32.00
60176,999WVJU110,Pulse,paid monthly,8.01,2022-09-14,2022-12-31,108,32.04


In [74]:
merge_columns = ['ID', 'BRAND', 'SUB_DETAIL', 'START_DATE', 'AMOUNT']
sub_rev_merged = pd.merge(sub_rev, sub_rev_2022, on=merge_columns, how='left', suffixes=('_full', '_2022'))
sub_rev_merged

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE_full,DURATION_full,REVENUE_full,END_DATE_2022,DURATION_2022,REVENUE_2022
0,000AHHU956,RetroReel,expired monthly,30.00,2022-01-01,2022-02-17,47,60.00,2022-02-17,47.0,60.00
1,000AHHU956,RetroReel,paid monthly,29.99,2022-05-21,2023-10-28,525,599.82,2022-12-31,224.0,239.92
2,000AHHU956,RetroReel,trial monthly,11.18,2023-11-05,2023-12-31,56,22.36,NaT,NaN,NaN
3,000BRHY264,CineQuest,paid annual,60.00,2022-05-23,2023-10-17,512,120.00,2022-12-31,222.0,60.00
4,000DJKK225,CineQuest,expired monthly,12.00,2022-01-01,2022-04-15,104,48.00,2022-04-15,104.0,48.00
...,...,...,...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,2023-10-01,2023-12-31,91,31.92,NaT,NaN,NaN
89541,999ZGDS170,CineQuest,expired annual,60.00,2022-01-01,2023-03-02,425,120.00,NaT,NaN,NaN
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,2023-12-31,39,59.99,NaT,NaN,NaN
89543,999ZIAU985,RetroReel,paid monthly,30.01,2022-01-28,2023-12-31,702,720.24,2022-12-31,337.0,360.12


In [75]:
def process_2022_data(row):
    if pd.isnull(row['END_DATE_2022']):
        # Case 1: `START_DATE` is in 2022, `END_DATE_full` is in 2023
        if row['START_DATE'] <= pd.to_datetime('2022-12-31') and row['END_DATE_full'] >= pd.to_datetime('2023-01-01'):
            row['END_DATE_2022'] = pd.to_datetime('2022-12-31')
            row['DURATION_2022'] = (pd.to_datetime('2022-12-31') - pd.to_datetime(row['START_DATE'])).days + 1
            if 'monthly' in row['SUB_DETAIL']:
                print(row['ID'])
                row['REVENUE_2022'] = np.ceil(row['DURATION_2022'] / 30) * row['AMOUNT']
            elif 'annual' in row['SUB_DETAIL']:
                print(row['ID'])
                row['REVENUE_2022'] = np.ceil(row['DURATION_2022'] / 365) * row['AMOUNT']

        # Case 2: Both `START_DATE` and `END_DATE_full` are in 2023
        elif row['START_DATE'] >= pd.to_datetime('2023-01-01'):
            row['END_DATE_2022'] = np.nan
            row['DURATION_2022'] = 0
            row['REVENUE_2022'] = 0
    return row

In [76]:
sub_rev_merged = sub_rev_merged.apply(process_2022_data, axis=1)
sub_rev_merged['END_DATE_2022'] = pd.to_datetime(sub_rev_merged['END_DATE_2022'])

# Calculate 2023 Revenue
sub_rev_merged['REVENUE_2023'] = sub_rev_merged['REVENUE_full'] - sub_rev_merged['REVENUE_2022']
sub_rev_merged['DURATION_2023'] = sub_rev_merged['DURATION_full'] - sub_rev_merged['DURATION_2022']
sub_rev_merged

000PGCL630
000TOJT688
001UAHA863
001WZZE262
002BKVK889
002FIHW100
003TEQR186
003XKHS955
003YXVZ587
003ZZGJ163
004SZWZ820
005AJTJ963
005CRSD874
005KLHD033
005UXRN097
007EBFN621
007IHUT323
008DEJF778
010ADQH166
010FMTD776
010FMTD776
010PKHU916
012FYQI592
012HJBP638
013QHTR180
014REWI075
015JHXW430
015PWDY650
016CMYY649
016FXDT429
016FXDT429
016YIWI121
017JQZU523
017ZMSQ311
018OBOB997
019PTUR768
022ODOM133
022WSQP330
023DHIY681
023MOVK325
023NUVZ956
024EAID291
025FQYG980
025LFEI109
026HNEL572
026IMJQ245
026SHHC102
026TCAF188
026XXER432
027SHZU688
028LYKE666
029MONS807
030JYTU571
030PDMK353
030XPXV557
031DDYY614
031EGOB154
031SBKW341
032DDSZ482
034YUAK722
036TGGJ227
036UGYQ840
038TZTM516
039EAWX176
039JTJQ019
040TNWD677
040WADJ377
041EKEP011
041QBSC636
042TDUA223
043HHHR895
043KBPY591
043YRUU646
044WWWO052
045QCOK064
045VZAO033
045ZMYZ414
046BCJW783
046UMHC124
047CTFE296
047GZWP978
047RTSI267
047XWVT596
048ANWV531
048BTCJ505
048JCYN921
048YGXX117
048YGXX117
049PJDN276
050RKOO458
053JWCW877

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE_full,DURATION_full,REVENUE_full,END_DATE_2022,DURATION_2022,REVENUE_2022,REVENUE_2023,DURATION_2023
0,000AHHU956,RetroReel,expired monthly,30.00,2022-01-01,2022-02-17,47,60.00,2022-02-17,47.0,60.00,0.00,0.0
1,000AHHU956,RetroReel,paid monthly,29.99,2022-05-21,2023-10-28,525,599.82,2022-12-31,224.0,239.92,359.90,301.0
2,000AHHU956,RetroReel,trial monthly,11.18,2023-11-05,2023-12-31,56,22.36,NaT,0.0,0.00,22.36,56.0
3,000BRHY264,CineQuest,paid annual,60.00,2022-05-23,2023-10-17,512,120.00,2022-12-31,222.0,60.00,60.00,290.0
4,000DJKK225,CineQuest,expired monthly,12.00,2022-01-01,2022-04-15,104,48.00,2022-04-15,104.0,48.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,2023-10-01,2023-12-31,91,31.92,NaT,0.0,0.00,31.92,91.0
89541,999ZGDS170,CineQuest,expired annual,60.00,2022-01-01,2023-03-02,425,120.00,2022-12-31,365.0,60.00,60.00,60.0
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,2023-12-31,39,59.99,NaT,0.0,0.00,59.99,39.0
89543,999ZIAU985,RetroReel,paid monthly,30.01,2022-01-28,2023-12-31,702,720.24,2022-12-31,337.0,360.12,360.12,365.0


In [81]:
sub_rev_merged.to_csv('/Users/vivianyan/Desktop/NYU_Capstone_Project/recalc_revenue/full_2022_2023_rev.csv')

In [ ]:
# Calculate 2023 Revenue
sub_rev_merged['REVENUE_2023'] = sub_rev_merged['REVENUE_full'] - sub_rev_merged['REVENUE_2022']
sub_rev_merged['DURATION_2023'] = sub_rev_merged['DURATION_full'] - sub_rev_merged['DURATION_2022']
sub_rev_merged

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE_full,DURATION_full,REVENUE_full,END_DATE_2022,DURATION_2022,REVENUE_2022,REVENUE_2023,DURATION_2023
0,000AHHU956,RetroReel,expired monthly,30.00,2022-01-01,2022-02-17,47,60.00,2022-02-17,47.0,60.00,0.00,0.0
1,000AHHU956,RetroReel,paid monthly,29.99,2022-05-21,2023-10-28,525,599.82,2022-12-31,224.0,239.92,359.90,301.0
2,000AHHU956,RetroReel,trial monthly,11.18,2023-11-05,2023-12-31,56,22.36,NaT,0.0,0.00,22.36,56.0
3,000BRHY264,CineQuest,paid annual,60.00,2022-05-23,2023-10-17,512,120.00,2022-12-31,222.0,60.00,60.00,290.0
4,000DJKK225,CineQuest,expired monthly,12.00,2022-01-01,2022-04-15,104,48.00,2022-04-15,104.0,48.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,2023-10-01,2023-12-31,91,31.92,NaT,0.0,0.00,31.92,91.0
89541,999ZGDS170,CineQuest,expired annual,60.00,2022-01-01,2023-03-02,425,120.00,2022-12-31,365.0,103.06,16.94,60.0
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,2023-12-31,39,59.99,NaT,0.0,0.00,59.99,39.0
89543,999ZIAU985,RetroReel,paid monthly,30.01,2022-01-28,2023-12-31,702,720.24,2022-12-31,337.0,360.12,360.12,365.0


In [82]:
sub_rev_2022 = sub_rev_merged[['ID', 'BRAND', 'SUB_DETAIL', 'AMOUNT', 'START_DATE', 'END_DATE_2022', 'DURATION_2022', 'REVENUE_2022']]
sub_rev_2022.rename(columns={'END_DATE_2022':'END_DATE', 'DURATION_2022':'DURATION', 'REVENUE_2022':'REVENUE'}, inplace=True)
sub_rev_2022

/var/folders/sv/tqkwxbbd1l96g8tr45l86pmw0000gn/T/ipykernel_85888/2773240989.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_rev_2022.rename(columns={'END_DATE_2022':'END_DATE', 'DURATION_2022':'DURATION', 'REVENUE_2022':'REVENUE'}, inplace=True)


,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
0,000AHHU956,RetroReel,expired monthly,30.00,2022-01-01,2022-02-17,47.0,60.00
1,000AHHU956,RetroReel,paid monthly,29.99,2022-05-21,2022-12-31,224.0,239.92
2,000AHHU956,RetroReel,trial monthly,11.18,2023-11-05,NaT,0.0,0.00
3,000BRHY264,CineQuest,paid annual,60.00,2022-05-23,2022-12-31,222.0,60.00
4,000DJKK225,CineQuest,expired monthly,12.00,2022-01-01,2022-04-15,104.0,48.00
...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,2023-10-01,NaT,0.0,0.00
89541,999ZGDS170,CineQuest,expired annual,60.00,2022-01-01,2022-12-31,365.0,60.00
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,NaT,0.0,0.00
89543,999ZIAU985,RetroReel,paid monthly,30.01,2022-01-28,2022-12-31,337.0,360.12


In [94]:
sub_rev_2022[sub_rev_2022['END_DATE'].isna()]['START_DATE'].dt.year.unique()

array([2023], dtype=int32)

In [96]:
sub_rev_2022.loc[sub_rev_2022['END_DATE'].isna(), 'START_DATE'] = pd.to_datetime(np.nan)
sub_rev_2022

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
0,000AHHU956,RetroReel,expired monthly,30.00,2022-01-01,2022-02-17,47.0,60.00
1,000AHHU956,RetroReel,paid monthly,29.99,2022-05-21,2022-12-31,224.0,239.92
2,000AHHU956,RetroReel,trial monthly,11.18,NaT,NaT,0.0,0.00
3,000BRHY264,CineQuest,paid annual,60.00,2022-05-23,2022-12-31,222.0,60.00
4,000DJKK225,CineQuest,expired monthly,12.00,2022-01-01,2022-04-15,104.0,48.00
...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,NaT,NaT,0.0,0.00
89541,999ZGDS170,CineQuest,expired annual,60.00,2022-01-01,2022-12-31,365.0,60.00
89542,999ZGDS170,CineQuest,paid annual,59.99,NaT,NaT,0.0,0.00
89543,999ZIAU985,RetroReel,paid monthly,30.01,2022-01-28,2022-12-31,337.0,360.12


In [97]:
sub_rev_2022.to_csv('/Users/vivianyan/Desktop/NYU_Capstone_Project/recalc_revenue/2022_rev.csv')

In [85]:
sub_rev_2023 = sub_rev_merged[['ID', 'BRAND', 'SUB_DETAIL', 'AMOUNT', 'START_DATE', 'END_DATE_full', 'DURATION_2023', 'REVENUE_2023']]
sub_rev_2023.rename(columns={'END_DATE_full':'END_DATE', 'DURATION_2023':'DURATION', 'REVENUE_2023':'REVENUE'}, inplace=True)
sub_rev_2023

/var/folders/sv/tqkwxbbd1l96g8tr45l86pmw0000gn/T/ipykernel_85888/1874367159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_rev_2023.rename(columns={'END_DATE_full':'END_DATE', 'DURATION_2023':'DURATION', 'REVENUE_2023':'REVENUE'}, inplace=True)


,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
0,000AHHU956,RetroReel,expired monthly,30.00,2022-01-01,2022-02-17,0.0,0.00
1,000AHHU956,RetroReel,paid monthly,29.99,2022-05-21,2023-10-28,301.0,359.90
2,000AHHU956,RetroReel,trial monthly,11.18,2023-11-05,2023-12-31,56.0,22.36
3,000BRHY264,CineQuest,paid annual,60.00,2022-05-23,2023-10-17,290.0,60.00
4,000DJKK225,CineQuest,expired monthly,12.00,2022-01-01,2022-04-15,0.0,0.00
...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,2023-10-01,2023-12-31,91.0,31.92
89541,999ZGDS170,CineQuest,expired annual,60.00,2022-01-01,2023-03-02,60.0,60.00
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,2023-12-31,39.0,59.99
89543,999ZIAU985,RetroReel,paid monthly,30.01,2022-01-28,2023-12-31,365.0,360.12


In [86]:
def process_2023_data(row):
    # Case 1: `START_DATE` is in 2022, `END_DATE_full` is in 2023
    if row['START_DATE'] < pd.to_datetime('2023-01-01') and row['END_DATE'] >= pd.to_datetime('2023-01-01'):
        row['START_DATE'] = pd.to_datetime('2023-01-01')

    # Case 2: Both `START_DATE` and `END_DATE_full` are in 2022
    elif row['START_DATE'] < pd.to_datetime('2023-01-01') and row['END_DATE'] < pd.to_datetime('2023-01-01'):
        row['START_DATE'] = pd.to_datetime(np.nan)
        row['END_DATE'] = pd.to_datetime(np.nan)
    return row

In [87]:
sub_rev_2023 = sub_rev_2023.apply(process_2023_data, axis=1)
sub_rev_2023

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
0,000AHHU956,RetroReel,expired monthly,30.00,NaT,NaT,0.0,0.00
1,000AHHU956,RetroReel,paid monthly,29.99,2023-01-01,2023-10-28,301.0,359.90
2,000AHHU956,RetroReel,trial monthly,11.18,2023-11-05,2023-12-31,56.0,22.36
3,000BRHY264,CineQuest,paid annual,60.00,2023-01-01,2023-10-17,290.0,60.00
4,000DJKK225,CineQuest,expired monthly,12.00,NaT,NaT,0.0,0.00
...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,2023-10-01,2023-12-31,91.0,31.92
89541,999ZGDS170,CineQuest,expired annual,60.00,2023-01-01,2023-03-02,60.0,60.00
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,2023-12-31,39.0,59.99
89543,999ZIAU985,RetroReel,paid monthly,30.01,2023-01-01,2023-12-31,365.0,360.12


In [98]:
sub_rev_2023.to_csv('/Users/vivianyan/Desktop/NYU_Capstone_Project/recalc_revenue/2023_rev.csv')

In [100]:
print(sub_rev_2023[sub_rev_2023['END_DATE'].isna()]['START_DATE'].unique(),
    sub_rev_2023[sub_rev_2023['START_DATE'].isna()]['END_DATE'].unique())

<DatetimeArray>
['NaT']
Length: 1, dtype: datetime64[ns] <DatetimeArray>
['NaT']
Length: 1, dtype: datetime64[ns]


In [146]:
def sep_2023_rev(row):
    threshold = pd.to_datetime('2023-06-30')

    if pd.isnull(row['START_DATE']):
        row['h1_start_date'] = pd.to_datetime(np.nan)
        row['h1_end_date'] = pd.to_datetime(np.nan)
        row['h1_duration'] = 0
        row['h1_rev'] = 0

        row['h2_start_date'] = pd.to_datetime(np.nan)
        row['h2_end_date'] = pd.to_datetime(np.nan)
        row['h2_duration'] = 0
        row['h2_rev'] = 0

    elif row['END_DATE'] <= threshold:
        row['h1_start_date'] = row['START_DATE']
        row['h1_end_date'] = row['END_DATE']
        row['h1_duration'] = row['DURATION']
        row['h1_rev'] = row['REVENUE']

        row['h2_start_date'] = pd.to_datetime(np.nan)
        row['h2_end_date'] = pd.to_datetime(np.nan)
        row['h2_duration'] = 0
        row['h2_rev'] = 0

    elif row['START_DATE'] > threshold:
        row['h1_start_date'] = pd.to_datetime(np.nan)
        row['h1_end_date'] = pd.to_datetime(np.nan)
        row['h1_duration'] = 0
        row['h1_rev'] = 0

        row['h2_start_date'] = row['START_DATE']
        row['h2_end_date'] = row['END_DATE']
        row['h2_duration'] = row['DURATION']
        row['h2_rev'] = row['REVENUE']

    elif row['START_DATE'] <= threshold and row['END_DATE'] > threshold:
        row['h1_start_date'] = row['START_DATE']
        row['h1_end_date'] = threshold
        row['h1_duration'] = (row['h1_end_date'] - row['h1_start_date']).days + 1
        row['h1_rev'] = (row['h1_duration'] / row['DURATION']) * row['REVENUE']
        # if 'monthly' in row['SUB_DETAIL']:
        #     row['h1_rev'] = np.ceil(row['h1_duration'] / 30) * row['AMOUNT']
        # elif 'annual' in row['SUB_DETAIL']:
        #     row['h1_rev'] = np.ceil(row['h1_duration'] / 365) * row['AMOUNT']

        row['h2_start_date'] = pd.to_datetime('2023-07-01')
        row['h2_end_date'] = row['END_DATE']
        row['h2_duration'] = row['DURATION'] - row['h1_duration']
        row['h2_rev'] = row['REVENUE'] - row['h1_rev']
    
    return row

In [147]:
sub_rev_2023 = sub_rev_2023.apply(sep_2023_rev, axis=1)
sub_rev_2023

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE,h1_start_date,h1_end_date,h1_duration,h1_rev,h2_start_date,h2_end_date,h2_duration,h2_rev
0,000AHHU956,RetroReel,expired monthly,30.00,NaT,NaT,0.0,0.00,NaT,NaT,0.0,0.000000,NaT,NaT,0.0,0.000000
1,000AHHU956,RetroReel,paid monthly,29.99,2023-01-01,2023-10-28,301.0,359.90,2023-01-01,2023-06-30,181.0,216.418272,2023-07-01,2023-10-28,120.0,143.481728
2,000AHHU956,RetroReel,trial monthly,11.18,2023-11-05,2023-12-31,56.0,22.36,NaT,NaT,0.0,0.000000,2023-11-05,2023-12-31,56.0,22.360000
3,000BRHY264,CineQuest,paid annual,60.00,2023-01-01,2023-10-17,290.0,60.00,2023-01-01,2023-06-30,181.0,37.448276,2023-07-01,2023-10-17,109.0,22.551724
4,000DJKK225,CineQuest,expired monthly,12.00,NaT,NaT,0.0,0.00,NaT,NaT,0.0,0.000000,NaT,NaT,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,2023-10-01,2023-12-31,91.0,31.92,NaT,NaT,0.0,0.000000,2023-10-01,2023-12-31,91.0,31.920000
89541,999ZGDS170,CineQuest,expired annual,60.00,2023-01-01,2023-03-02,60.0,60.00,2023-01-01,2023-03-02,60.0,60.000000,NaT,NaT,0.0,0.000000
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,2023-12-31,39.0,59.99,NaT,NaT,0.0,0.000000,2023-11-22,2023-12-31,39.0,59.990000
89543,999ZIAU985,RetroReel,paid monthly,30.01,2023-01-01,2023-12-31,365.0,360.12,2023-01-01,2023-06-30,181.0,178.580055,2023-07-01,2023-12-31,184.0,181.539945


In [148]:
sub_rev_2023_h1 = sub_rev_2023[['ID', 'BRAND', 'SUB_DETAIL', 'AMOUNT', 'h1_start_date', 'h1_end_date', 'h1_duration', 'h1_rev']]
sub_rev_2023_h1.rename(columns={'h1_start_date':'START_DATE', 'h1_end_date':'END_DATE', 'h1_duration':'DURATION', 'h1_rev':'REVENUE'}, inplace=True)
sub_rev_2023_h1

/var/folders/sv/tqkwxbbd1l96g8tr45l86pmw0000gn/T/ipykernel_85888/3712657884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_rev_2023_h1.rename(columns={'h1_start_date':'START_DATE', 'h1_end_date':'END_DATE', 'h1_duration':'DURATION', 'h1_rev':'REVENUE'}, inplace=True)


,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
0,000AHHU956,RetroReel,expired monthly,30.00,NaT,NaT,0.0,0.000000
1,000AHHU956,RetroReel,paid monthly,29.99,2023-01-01,2023-06-30,181.0,216.418272
2,000AHHU956,RetroReel,trial monthly,11.18,NaT,NaT,0.0,0.000000
3,000BRHY264,CineQuest,paid annual,60.00,2023-01-01,2023-06-30,181.0,37.448276
4,000DJKK225,CineQuest,expired monthly,12.00,NaT,NaT,0.0,0.000000
...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,NaT,NaT,0.0,0.000000
89541,999ZGDS170,CineQuest,expired annual,60.00,2023-01-01,2023-03-02,60.0,60.000000
89542,999ZGDS170,CineQuest,paid annual,59.99,NaT,NaT,0.0,0.000000
89543,999ZIAU985,RetroReel,paid monthly,30.01,2023-01-01,2023-06-30,181.0,178.580055


In [149]:
sub_rev_2023_h2 = sub_rev_2023[['ID', 'BRAND', 'SUB_DETAIL', 'AMOUNT', 'h2_start_date', 'h2_end_date', 'h2_duration', 'h2_rev']]
sub_rev_2023_h2.rename(columns={'h2_start_date':'START_DATE', 'h2_end_date':'END_DATE', 'h2_duration':'DURATION', 'h2_rev':'REVENUE'}, inplace=True)
sub_rev_2023_h2

/var/folders/sv/tqkwxbbd1l96g8tr45l86pmw0000gn/T/ipykernel_85888/734778524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_rev_2023_h2.rename(columns={'h2_start_date':'START_DATE', 'h2_end_date':'END_DATE', 'h2_duration':'DURATION', 'h2_rev':'REVENUE'}, inplace=True)


,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
0,000AHHU956,RetroReel,expired monthly,30.00,NaT,NaT,0.0,0.000000
1,000AHHU956,RetroReel,paid monthly,29.99,2023-07-01,2023-10-28,120.0,143.481728
2,000AHHU956,RetroReel,trial monthly,11.18,2023-11-05,2023-12-31,56.0,22.360000
3,000BRHY264,CineQuest,paid annual,60.00,2023-07-01,2023-10-17,109.0,22.551724
4,000DJKK225,CineQuest,expired monthly,12.00,NaT,NaT,0.0,0.000000
...,...,...,...,...,...,...,...,...
89540,999WVJU110,Pulse,paid monthly,7.98,2023-10-01,2023-12-31,91.0,31.920000
89541,999ZGDS170,CineQuest,expired annual,60.00,NaT,NaT,0.0,0.000000
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,2023-12-31,39.0,59.990000
89543,999ZIAU985,RetroReel,paid monthly,30.01,2023-07-01,2023-12-31,184.0,181.539945


In [152]:
sub_rev_2023_h1.to_csv('/Users/vivianyan/Desktop/NYU_Capstone_Project/recalc_revenue/2023_first_6m_rev.csv')
sub_rev_2023_h2.to_csv('/Users/vivianyan/Desktop/NYU_Capstone_Project/recalc_revenue/2023_last_6m_rev.csv')

In [153]:
sub_rev_2023_h1_cleaned = sub_rev_2023_h1.dropna()
sub_rev_2023_h2_cleaned = sub_rev_2023_h2.dropna()

In [154]:
sub_rev_2023_h1_cleaned.to_csv('/Users/vivianyan/Desktop/NYU_Capstone_Project/recalc_revenue/dropna_2023_first_6m_rev.csv')
sub_rev_2023_h2_cleaned.to_csv('/Users/vivianyan/Desktop/NYU_Capstone_Project/recalc_revenue/dropna_2023_last_6m_rev.csv')

In [155]:
sub_rev_2023_h1_cleaned

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
1,000AHHU956,RetroReel,paid monthly,29.99,2023-01-01,2023-06-30,181.0,216.418272
3,000BRHY264,CineQuest,paid annual,60.00,2023-01-01,2023-06-30,181.0,37.448276
5,000DJKK225,CineQuest,paid monthly,12.00,2023-01-01,2023-06-30,181.0,71.408219
8,000EFUI271,LimeLight,paid annual,80.01,2023-01-01,2023-06-30,181.0,39.676192
9,000ELOE240,LimeLight,trial annual,51.62,2023-01-01,2023-04-25,115.0,0.000000
...,...,...,...,...,...,...,...,...
89537,999WMMD928,LimeLight,paid annual,80.00,2023-01-01,2023-06-30,181.0,39.671233
89539,999WVJU110,Pulse,paid monthly,8.01,2023-01-01,2023-06-30,181.0,73.377553
89541,999ZGDS170,CineQuest,expired annual,60.00,2023-01-01,2023-03-02,60.0,60.000000
89543,999ZIAU985,RetroReel,paid monthly,30.01,2023-01-01,2023-06-30,181.0,178.580055


# Test

In [12]:
sub_rev_2022[sub_rev_2022.SUB_DETAIL.str.contains('discount')]

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
17,000ORNZ823,CineQuest,discounted monthly,6.78,2022-05-01,2022-10-07,159,88.72
44,000VITQ690,CineQuest,discounted monthly,5.18,2022-08-19,2022-12-31,134,25.90
62,001JDME989,LimeLight,discounted annual,76.56,2022-12-10,2022-12-31,21,76.56
83,001SIVU071,ChillStream,discounted annual,36.56,2022-06-03,2022-09-10,99,36.56
99,001UFAQ102,CineQuest,discounted annual,27.20,2022-12-21,2022-12-31,10,27.20
...,...,...,...,...,...,...,...,...
60083,998NTVU114,LimeLight,discounted annual,35.67,2022-02-09,2022-12-31,325,35.67
60122,999ERRR461,CineQuest,discounted monthly,10.42,2022-10-12,2022-12-31,80,31.26
60142,999LAFI268,LimeLight,discounted annual,41.79,2022-01-26,2022-07-06,161,41.79
60153,999PNJX936,Pulse,discounted monthly,4.42,2022-04-24,2022-06-24,61,29.24


In [25]:
sub_rev_2022[sub_rev_2022.ID=='999ZGDS170']

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE


In [26]:
sub_rev[sub_rev.ID=='999ZGDS170']

,ID,BRAND,SUB_DETAIL,AMOUNT,START_DATE,END_DATE,DURATION,REVENUE
89541,999ZGDS170,CineQuest,expired annual,60.00,2022-01-01,2023-03-02,425,120.00
89542,999ZGDS170,CineQuest,paid annual,59.99,2023-11-22,2023-12-31,39,59.99


In [27]:
sub_filtered[sub_filtered.ID=='999ZGDS170']

,Unnamed: 0,ID,BRAND,SUB_STATUS,TERM,AMOUNT,PARTITION_DATE,YEAR_MONTH,VALID_TRANSITION
132581,219210,999ZGDS170,CineQuest,expired,annual,60.00,2023-03-02,2023-03,True
132582,219211,999ZGDS170,CineQuest,discounted,annual,37.46,2023-07-28,2023-07,True
132583,219213,999ZGDS170,CineQuest,paid,annual,59.99,2023-11-22,2023-11,True
